# Setting

- We have k chunks to estimate
- Each chunk i already has $m_i$ past results $R_{ij} \sim Lap(1/n_i \epsilon_{ij})$
- Linear combination: $Q_{ij} := \frac{n_i}{n} \gamma_{ij}(q_i + R_{ij})$
- We need $\sum_j \gamma_{ij} = 1$ for each chunk $i$ to have an unbiased estimate
- Goal: output an estimate $Q = Q_{10} + \dots + Q_{1m_1} +\dots + Q_{k0} + \dots + Q_{km_k}$
- Accuracy constraint: $\Pr[|Q-q|>\alpha] < \beta$
- Simplifying budget constraint: $R_{i0} \sim Lap(1/n_\epsilon)$ (same budget for everyone!). That's a weird constraint if we have chunks that are already good, let's see if we can set some epsilons to infinity. In the general case we could try an ILP or even some gradient descent on the budget vector?

In [1]:
import numpy as np

In [2]:
def minimum_variance(epsilons, noises, chunk_sizes):
    n_chunks = len(epsilons)
    chunk_noises = np.zeros(n_chunks)
    for chunk_id in range(n_chunks):
        # The variance has an extra 2/n_i^2 factor but it doesn't matter at the chunk level
        laplace_coefficients = epsilons[chunk_id] ** 2
        # See optimal variance reduction and lemma in Overleaf
        laplace_coefficients = laplace_coefficients / sum(laplace_coefficients)
        chunk_noises[chunk_id] = np.dot(laplace_coefficients, noises[chunk_id])
        
    chunk_coefficients = chunk_sizes / sum(chunk_sizes)
    aggregated_noise_total = np.dot(chunk_coefficients, chunk_noises)
    return aggregated_noise_total

In [25]:
# TODO: given epsilon and past results, find beta

def monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=1_000_000):
    
    # Add fresh epsilon
    epsilons = [
        np.append(eps_by_chunk, fresh_epsilon) for eps_by_chunk in existing_epsilons
    ]
    
    # TODO: heuristic to drop some chunks?
    
    # Vectorized code with a batch dimension corresponding to N
    n_chunks = len(epsilons)
    n = sum(chunk_sizes)
    chunk_noises = np.zeros((N, n_chunks))
    for chunk_id in range(n_chunks):
        # The final laplace scale (Q_ij), already scaled by n_i/n * eps^2/sum(eps^2)
        single_chunk_laplace_scale = epsilons[chunk_id] / (n * np.sum(epsilons[chunk_id] ** 2))
        laplace_scale = np.repeat([single_chunk_laplace_scale], N, axis=0)
        laplace_noises = np.random.laplace(scale=laplace_scale)
        
        
        # Optimal average for that chunk, N times
        chunk_noises[:, chunk_id] = np.sum(laplace_noises, axis=1)
        
    aggregated_noise_total = np.sum(chunk_noises, axis=1)
    beta = np.sum(aggregated_noise_total > alpha) / N
    return beta


In [31]:
existing_epsilons = [
    np.array([1, 0.5, 2]),
    np.array([0.3, 2.1])
]

chunk_sizes = [100, 200]
fresh_epsilon = 0.1
alpha = 0.001

monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=10_000_000)

0.3511366

In [3]:
# TODO: binary search for epsilon